In [1]:
# -*- coding: UTF-8 -*-
"""Evaluation performance of the most popular classification algorithms with
    artificial dataset generated by sklearn.datasets.make_classification()"""
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_validate
from sklearn import metrics
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier, HistGradientBoostingClassifier, RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [2]:
# Define test classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=2, random_state=12)
print(f"Summarize the dataset: X.shape is {X.shape}; y.shape is {y.shape}")
# Split for train/test data as 80% and 20 %
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

Summarize the dataset: X.shape is (1000, 10); y.shape is (1000,)


In [3]:
# Instances of the classification models
lr = LogisticRegression()
rf = RandomForestClassifier()
gb = GradientBoostingClassifier()
hist_gb = HistGradientBoostingClassifier()
xgb = XGBClassifier()
lgb = LGBMClassifier()
catgb = CatBoostClassifier(verbose=0, n_estimators=200)
mlp = MLPClassifier(max_iter=1500, solver='adam')

models = (lr, rf, gb, hist_gb, xgb, lgb, catgb, mlp)
models_name = ('LogisticRegression',
                'RandomForestClassifier',
                'GradientBoostingClassifier',
                'HistGradientBoostingClassifier',
                'XGBClassifier',
                'LGBMClassifier',
                'CatBoostClassifier',
                'MLPClassifier',
               )
models

(LogisticRegression(),
 RandomForestClassifier(),
 GradientBoostingClassifier(),
 HistGradientBoostingClassifier(),
 XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, gamma=None,
               gpu_id=None, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=None, max_bin=None,
               max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
               max_leaves=None, min_child_weight=None, missing=nan,
               monotone_constraints=None, n_estimators=100, n_jobs=None,
               num_parallel_tree=None, predictor=None, random_state=None,
               reg_alpha=None, reg_lambda=None, ...),
 LGBMClassifier(),
 MLPClassifier(max_iter=1500))

In [4]:
def make_cross_validation(estimator, X_train, y_train, cv=5):
    """Perform cross validation and return metrics:
    accuracy, balanced_accuracy, f1, precision, recall, roc_auc"""
    scorings = ('accuracy', 'balanced_accuracy', 'f1', 'precision', 'recall', 'roc_auc')
    scores = cross_validate(estimator, X_train, y_train, cv=cv, n_jobs=-1, scoring=scorings)
    final_metrics = dict()
    for key, item in sorted(scores.items()):
        final_metrics[key] = item.mean().round(4)
    return final_metrics

In [5]:
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=12)
data_cross_val = []
for mdl in models:
    report_dict = make_cross_validation(mdl, X_train, y_train, cv=cv)
    data_cross_val.append(report_dict)
df_report = pd.DataFrame(data_cross_val, index=models_name)
df_report

,fit_time,score_time,test_accuracy,test_balanced_accuracy,test_f1,test_precision,test_recall,test_roc_auc
LogisticRegression,0.0147,0.0085,0.8325,0.8323,0.8362,0.8291,0.8449,0.9049
RandomForestClassifier,0.7780,0.0864,0.8982,0.8982,0.9001,0.8962,0.9047,0.9672
GradientBoostingClassifier,0.7496,0.0156,0.8988,0.8986,0.9009,0.8946,0.9081,0.9582
HistGradientBoostingClassifier,0.5964,0.0298,0.9278,0.9278,0.9285,0.9309,0.9269,0.9721
XGBClassifier,0.4513,0.0269,0.9225,0.9224,0.9240,0.9198,0.9289,0.9711
LGBMClassifier,0.2448,0.0267,0.9238,0.9238,0.9247,0.9264,0.9235,0.9711
CatBoostClassifier,2.4718,0.0103,0.9302,0.9301,0.9317,0.9247,0.9393,0.9745
MLPClassifier,7.2410,0.0119,0.9322,0.9322,0.9335,0.9298,0.9378,0.9761


In [6]:
def test_evaluation(estimator, X_train, y_train, X_test, y_test) -> dict:
    """Calculate the main model metrics - accuracy, balanced-Accuracy,
    recall, precision, f1_score on the test set and return them as a map object."""
    estimator.fit(X_train, y_train)
    y_pred = estimator.predict(X_test)

    accuracy = metrics.accuracy_score(y_test, y_pred).round(4)
    balanced_accuracy = metrics.balanced_accuracy_score(y_test, y_pred).round(4)
    precision = metrics.precision_score(y_test, y_pred).round(4)
    f_1 = metrics.f1_score(y_test, y_pred).round(4)
    recall = metrics.recall_score(y_test, y_pred).round(4)
    roc_auc = metrics.roc_auc_score(y_test, y_pred).round(4)

    final_metrics = {'accuracy': accuracy,
                     'balanced_accuracy': balanced_accuracy,
                     'f1': f_1,
                     'precision': precision,
                     'recall': recall,
                     'roc_auc': roc_auc,
                    }
    # Use dict comprehension for round dict values
    final_metrics = {key: round(val, 4) for key, val in final_metrics.items()}
    return final_metrics

In [7]:
# Evaluate the models on the test set
data_test = []
for mdl in models:
    report_dict_test = test_evaluation(mdl, X_train, y_train, X_test, y_test)
    data_test.append(report_dict_test)
print('Evaluate the models on the test set: ')
df_report_test = pd.DataFrame(data_test, index=models_name)
df_report_test

Evaluate the models on the test set: 


,accuracy,balanced_accuracy,f1,precision,recall,roc_auc
LogisticRegression,0.845,0.8438,0.8324,0.8851,0.7857,0.8438
RandomForestClassifier,0.915,0.9145,0.9110,0.9355,0.8878,0.9145
GradientBoostingClassifier,0.890,0.8898,0.8866,0.8958,0.8776,0.8898
HistGradientBoostingClassifier,0.940,0.9400,0.9388,0.9388,0.9388,0.9400
XGBClassifier,0.920,0.9196,0.9167,0.9362,0.8980,0.9196
LGBMClassifier,0.930,0.9300,0.9286,0.9286,0.9286,0.9300
CatBoostClassifier,0.950,0.9498,0.9485,0.9583,0.9388,0.9498
MLPClassifier,0.945,0.9449,0.9436,0.9485,0.9388,0.9449
